In [38]:
import torch
from datasets import load_dataset

In [3]:
dataset = load_dataset('oscar', "unshuffled_deduplicated_ru", split='train', streaming=True)

In [5]:
next(iter(dataset))

{'id': 0,
 'text': 'Если ваши поиски бесплатных объявлений в разделе «Настольные компьютеры» для города Нижегородская область не привели к желаемому результату,'}

In [7]:
for i, data in enumerate(dataset):
    print(i, data)
    if i == 10:
        break

0 {'id': 0, 'text': 'Если ваши поиски бесплатных объявлений в разделе «Настольные компьютеры» для города Нижегородская область не привели к желаемому результату,'}
1 {'id': 1, 'text': 'Доступ к данному профилю для публичного просмотра закрыт администрацией сайта - профиль находится на модерации.\nРазработчикам | Служба технической поддержки | Пользовательское соглашение | Реклама | Главная KM.RU\nСамая популярная Украинская платформа для поиска репетиторов по любому предмету на всей территории Украины и по Skype!\nУ нас есть возможность подобрать именно того репетитора, который специализируется на решении актуальных для вас задач.\nЕсли вы не нашли подходящего репетитора, доверьтесь нам и мы порекомендуем вам лучших специалистов, которые соответствуют всем вашим требованиям.\nНаши консультации и подбор подходящего вам репетитора - бесплатны. Вы оплачиваете только сами занятия, непосредственно репетитору.\nДля вас мы проведём поиск подходящего репетитора по всей нашей базе, которая соде

In [8]:
shuffled_dataset = dataset.shuffle(seed=42, buffer_size=100)

In [9]:
next(iter(shuffled_dataset))

{'id': 8,
 'text': 'Шаловливая, горячая развратница!Дыхание страсти опалит твой разум, ты позабудешь обо всем в моих объятиях!\nМиловидная куколка, в 40 лет в жопу новосибирск, жаждущая страсти Самый теплый прием Я встречу тебя танцем, после которого ты просто не сможешь устоять Звоночки принимаю в любое время. Метки ролевые игры в Красноуфимске, девушка в Красноуфимске, услуги семейной паре в Красноуфимске. И вот с недельку назад приехал в гости один знакомый, ну и решили мы, как говорят шведы, попробовать молодого мяса Поскольку со Щербинкой у него связаны только хорошие ассоциации, попросил меня съездить туда Ну что не сделаешь для ближнего Взял такси так как выпили за встречуи поехал, хотя чувствовал, что еду туда наверно зря. Страна Россия Регион Воронежская область Город Воронеж. Услуги Минет без презерватива, Лейсбийский, Двойное проникновение. МИЛАЯ, ОБЩИТЕЛЬНАЯ ДЕВОЧКА ПРИГЛАСИТ В ГОСТИ, А ТАК ЖЕ ПРИЕДЕТ В ГОСТИ СКРАСИТЬ ВАШ ДОСУГ СО МНОЙ ТЫ ЗАБУДЕШЬ В 40 лет в жопу новосибирс

In [10]:
for i, data in enumerate(shuffled_dataset):
    print(i, data)
    if i == 10:
        break

0 {'id': 8, 'text': 'Шаловливая, горячая развратница!Дыхание страсти опалит твой разум, ты позабудешь обо всем в моих объятиях!\nМиловидная куколка, в 40 лет в жопу новосибирск, жаждущая страсти Самый теплый прием Я встречу тебя танцем, после которого ты просто не сможешь устоять Звоночки принимаю в любое время. Метки ролевые игры в Красноуфимске, девушка в Красноуфимске, услуги семейной паре в Красноуфимске. И вот с недельку назад приехал в гости один знакомый, ну и решили мы, как говорят шведы, попробовать молодого мяса Поскольку со Щербинкой у него связаны только хорошие ассоциации, попросил меня съездить туда Ну что не сделаешь для ближнего Взял такси так как выпили за встречуи поехал, хотя чувствовал, что еду туда наверно зря. Страна Россия Регион Воронежская область Город Воронеж. Услуги Минет без презерватива, Лейсбийский, Двойное проникновение. МИЛАЯ, ОБЩИТЕЛЬНАЯ ДЕВОЧКА ПРИГЛАСИТ В ГОСТИ, А ТАК ЖЕ ПРИЕДЕТ В ГОСТИ СКРАСИТЬ ВАШ ДОСУГ СО МНОЙ ТЫ ЗАБУДЕШЬ В 40 лет в жопу новосибир

In [12]:
from transformers import BloomTokenizerFast, BloomForCausalLM

In [13]:
%%time
tokenizer = BloomTokenizerFast.from_pretrained("bigscience/bloom-560m")
model = BloomForCausalLM.from_pretrained("bigscience/bloom-560m")

CPU times: user 4.9 s, sys: 1.26 s, total: 6.16 s
Wall time: 10.2 s


In [39]:
def encode(examples):
    inputs = tokenizer(examples['text'], padding=True, return_tensors='pt')
    inputs['labels'] = torch.where(inputs['input_ids'] == 3, -100, inputs['input_ids'])
    return inputs

In [40]:
dataset = load_dataset('oscar', "unshuffled_deduplicated_ru", split='train', streaming=True)
dataset = dataset.map(encode, batched=True, remove_columns=['id', 'text'], batch_size=2)

In [41]:
next(iter(dataset))

{'input_ids': tensor([    3,     3,     3,  ..., 24150, 93802,    15]),
 'attention_mask': tensor([0, 0, 0,  ..., 1, 1, 1]),
 'labels': tensor([ -100,  -100,  -100,  ..., 24150, 93802,    15])}

In [42]:
for i, data in enumerate(dataset):
    print(i, data)
    print('-'*100)
    if i == 1:
        break

0 {'input_ids': tensor([    3,     3,     3,  ..., 24150, 93802,    15]), 'attention_mask': tensor([0, 0, 0,  ..., 1, 1, 1]), 'labels': tensor([ -100,  -100,  -100,  ..., 24150, 93802,    15])}
----------------------------------------------------------------------------------------------------
1 {'input_ids': tensor([129144,  41160, 141591,  ...,  41787,   9544,     17]), 'attention_mask': tensor([1, 1, 1,  ..., 1, 1, 1]), 'labels': tensor([129144,  41160, 141591,  ...,  41787,   9544,     17])}
----------------------------------------------------------------------------------------------------


In [ ]:
texts

In [57]:
dataset = load_dataset('nthngdy/oscar-mini', 'unshuffled_deduplicated_ru', split='train')

Found cached dataset oscar-mini (/home/alexnikko/.cache/huggingface/datasets/nthngdy___oscar-mini/unshuffled_deduplicated_ru/1.0.0/d61b181331745a38dd31e8c6cc23d46566b96e255384c4421f2396af24a01dff)


In [58]:
def collate_fn(data):
    texts = [x['text'] for x in data]
    inputs = tokenizer(texts, padding=True, return_tensors='pt')
    inputs['labels'] = torch.where(inputs['input_ids'] == 3, -100, inputs['input_ids'])
    return inputs

In [59]:
from torch.utils.data import DataLoader

In [60]:
loader = DataLoader(dataset, collate_fn=collate_fn, batch_size=2)

In [61]:
batch = next(iter(loader))
batch

{'input_ids': tensor([[184840, 159460, 223439,  51767,  64494,   2425,  59009,  19192, 143519,
          20064,   2568, 103043, 248425,   5903,   5375,   4486,  28741,   9721,
          11536,  76099,  49673,  40359,   1871, 106429,   1962,   8337,  61000,
          83089, 135729,  10057,   5824, 188368,  21789,   7057,   2361, 125927,
         247397, 135658, 195646, 166205,  12099,  20243,  21183, 151158,  53333,
          20064, 107952,  40198,  52745,  54168,  16193,  15740, 207996,  20064,
          23072,  22459,   5252,  76471,   9231,   5252,  25419,  24150,  93802,
             15],
        [     3,      3,      3, 101024,   5252,  66982,  10244,    602,   2906,
           2829,  48405,  17597,   9231,  54994,  12248,  67526,  31735, 178699,
           9544, 124059,    602,  31996,    602,  57606,    620,     15, 116653,
           2245,  13922, 206043,    915,  48405, 184840, 106429, 211373, 131415,
         106429, 125475,  71929,    142,    108, 184840,  59442, 184840, 1064

In [62]:
outputs = model(**batch)

In [63]:
loss = outputs.loss
logits = outputs.logits
perplexity = torch.exp(loss)

In [64]:
print(f'Loss = {loss.item()}')
print(f'Perplexity = {perplexity.item()}')

Loss = 3.7054500579833984
Perplexity = 40.6683464050293
